# Forêt d'arbres décisionnels (Random forest)

Implémentation d'une forêt d'arbres décisionnels avec une recherche d'hyper-paramètres.

## Introduction

- *Objectif*: Détecter la pneumonie à partir des images radiographiques à l'aide du ML
- *Datasets*: Train, Validation & Test des ensembles d'images JPEG

L'objectif de ce projet est de développer un modèle de Machine Learning pour détecter la pneumonie à partir des images radiographiques.

# Etapes

1. **Chargement des données** à partir des dossiers déjà splités (train/test)
2. **Pré-traîtement** des données
3. **Initialisation du modèle**.
4. **Recherche d'hyper-paramètres**.
5. **Evaluation du modèle** et **prédiction**.

### Import de libraries nécessaires

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score

## I. Chargement et split des données en fonction du dossier

### Définir les paths des ensembles de données

*(charger des ensembles de données et explorer sa structure)*

Les images sont redimensionnées à 64x64px pour avoir une taille uniforme et plus légère

In [16]:
import numpy as np

from preprocessing import load_images_from_folder

import os

base_path = "/Users/minhduc/Documents/MSc1/T-DEV-810/chest_Xray"

train_path = os.path.join(base_path, "train")
test_path = os.path.join(base_path, "test")

X_train, Y_train = load_images_from_folder(train_path, image_size=(64, 64))
X_test, Y_test = load_images_from_folder(test_path, image_size=(64, 64))

## II. Pré-traitement des données

- Augmentation des données pour avoir une distribution des classes homogènes
- Normalisation des images pour "rescaler" les valeurs des pixels dans une page uniforme
- Aplanissement des images (2D -> 1D) pour les rendre compatibles

In [17]:
from src.augment import augment_images_to_balance

X_train_augmented, Y_train_augmented = augment_images_to_balance(X_train, Y_train)

flattened_images = np.array([x.flatten() for x in X_train_augmented])
test_flattened_images = np.array([x.flatten() for x in X_test])

## III. Initialisation du modèle et recherche des hyperparamètres optimaux du modèle avec les données traitées

- Initialisation du modèle basé sur un arbre de décision.
- Définition de la grille de recherche d'hyper-paramètres.
- Recherche d'hyper-paramètres avec GridSearchCV basée sur la validation croisée.
- Entraînement et recherche d'hyper-paramètres

In [18]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 'log2'],
}

rf_clf = RandomForestClassifier(random_state=0)

grid_search = GridSearchCV(rf_clf, param_grid, cv=5, n_jobs=-1)
grid_search.fit(flattened_images, Y_train_augmented)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]})

### Affichage des hyperparamètres calculés par la recherche

In [19]:
best_params = grid_search.best_params_

## IV. Evaluation du meilleur modèle selectionné

- Validation croisée pour évaluer les performance du modèle.
- Prédiction et évaluation du modèle sur les données de test.

In [20]:
best_clf = grid_search.best_estimator_
scores = cross_val_score(best_clf, flattened_images, Y_train_augmented, cv=5, scoring='accuracy')

print(f"Scores de validation croisée : {scores}")
print(f"Score moyen de validation croisée : {np.mean(scores)}")

Y_prediction = best_clf.predict(test_flattened_images)

print(f"Accuracy sur les données de test : {accuracy_score(Y_test, Y_prediction)}")
print(f"Rapport de classification :\n {classification_report(Y_test, Y_prediction)}")

Scores de validation croisée : [0.8716129  0.93032258 0.98129032 0.98967742 0.97935484]
Score moyen de validation croisée : 0.9504516129032258
Accuracy sur les données de test : 0.8044871794871795
Rapport de classification :
               precision    recall  f1-score   support

           0       0.95      0.50      0.66       234
           1       0.77      0.98      0.86       390

    accuracy                           0.80       624
   macro avg       0.86      0.74      0.76       624
weighted avg       0.84      0.80      0.79       624
